## Import the modules

### Basic imports

In [ ]:
import os
from tqdm.notebook import tqdm

import numpy as np 
import pandas as pd 
from collections import Counter

import matplotlib
from matplotlib import pyplot as plt

import PIL
from PIL import Image
from IPython.display import display

import openslide
import skimage.io

from sklearn.model_selection import train_test_split

import cv2
import zipfile

### Model based imports

In [ ]:
import tensorflow as tf
import keras
from keras import Model
from keras.layers import Input, Dropout, concatenate, Dense
from keras.layers.convolutional import Conv2D, MaxPooling2D, UpSampling2D

import keras.backend as K
from keras.preprocessing.image import ImageDataGenerator

from keras import models
from keras import layers
from keras import optimizers

## Data folders

In [ ]:
BASE_DIR = '../input/prostate-cancer-grade-assessment/'
SAVE_DIR = '/kaggle/working/'

OVERLAY_IMG_DIR = '/kaggle/working/overlay/'
TILING_IMG_DIR = '/kaggle/working/tiled_images/'

IMG_SIZE = 224

# The Dataset

In [ ]:
base_data = pd.read_csv(BASE_DIR+'train.csv')

## Filtering non-mask images

In [ ]:
images_without_mask = []
for image in base_data['image_id']:
    if not os.path.exists(BASE_DIR+'train_label_masks/'+image+'_mask.tiff'):
        images_without_mask.append(image)

data_without_mask = base_data[base_data['image_id'].isin(images_without_mask)]

base_data = base_data[~base_data['image_id'].isin(images_without_mask)]

## Separate the data sources

In [ ]:
radboud_train_set = base_data[base_data['data_provider']=='radboud']
karolinska_train_set = base_data[base_data['data_provider']=='karolinska']

# Step 1: Segmentation Model

### Split and load the data for the segmentation model

In [ ]:
def load_images(image, with_mask=True):
    
    slide = openslide.OpenSlide(os.path.join(BASE_DIR+"train_images", f'{image}.tiff'))
    
    spacing = 1 / (float(slide.properties['tiff.XResolution']) / 10000)
    img = slide.get_thumbnail(size=(IMG_SIZE,IMG_SIZE))
    
    
    img = Image.fromarray(np.array(img))
    img = img.resize((IMG_SIZE, IMG_SIZE))
    img = np.array(img)

    if with_mask:
        mask =  openslide.OpenSlide(os.path.join(BASE_DIR+'train_label_masks', f'{image}_mask.tiff'))
        mask_data = mask.read_region((0,0), mask.level_count - 1, mask.level_dimensions[-1])

        mask_data = Image.fromarray(np.array(mask_data))
        mask_data = mask_data.resize((IMG_SIZE, IMG_SIZE))
        mask_data = np.array(mask_data)
        mask_data = mask_data/5
    
        return img, mask_data[:,:,0]

    return img

In [ ]:
unet_train_ids, unet_test_ids, unet_train_labels, unet_test_labels = train_test_split(radboud_train_set['image_id'], radboud_train_set['isup_grade'], train_size=0.85)

### Loss metric: Dice coefficient

In [ ]:
def Dice_coeff(y_true, y_pred):
    smooth = 1
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

### UNet model

In [ ]:
def unet_model():
    in1 = Input(shape=(IMG_SIZE, IMG_SIZE, 3 ))

    conv1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(in1)
    conv1 = Dropout(0.3)(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool1)
    conv2 = Dropout(0.3)(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool2)
    conv3 = Dropout(0.3)(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)
    
    
    conv4 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool3)
    conv4 = Dropout(0.3)(con43)
    conv4 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)
    
    conv5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool4)
    conv5 = Dropout(0.3)(conv5)
    conv5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv5)
    up1 = concatenate([UpSampling2D((2, 2))(conv5), conv4], axis=-1)

    conv6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool4)
    conv6 = Dropout(0.3)(conv5)
    conv6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv5)

    up1 = concatenate([UpSampling2D((2, 2))(conv5), conv3], axis=-1)
    conv6 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up1)
    conv6 = Dropout(0.3)(conv6)
    conv6 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv6)
    
    up2 = concatenate([UpSampling2D((2, 2))(conv5), conv2], axis=-1)
    conv7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up2)
    conv7 = Dropout(0.3)(conv6)
    conv7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv6)

    up3 = concatenate([UpSampling2D((2, 2))(conv6), conv1], axis=-1)
    conv8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up3)
    conv8 = Dropout(0.3)(conv7)
    conv8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv8)
    segmentation = Conv2D(1, (1, 1), activation='sigmoid', name='seg')(conv8)

    model = Model(inputs=[in1], outputs=[segmentation])
    print(model.summary())
    
    optimizer=keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=0.001, decay=0.0, amsgrad=True)
    
    model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['acc', Dice_coeff])
    
    return model

In [ ]:
seq_model = unet_model()

### Model checkpoint and Early Stopping

In [ ]:
from keras.callbacks import EarlyStopping,ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)
mc = ModelCheckpoint(SAVE_DIR+'Segmentor_checkpoint.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

In [ ]:
unet_train_X = []
unet_train_Y = []

for img_id in tqdm(unet_train_ids):
    train_img, msk_img = load_images(img_id)
    unet_train_X.append(train_img)
    unet_train_Y.append(msk_img)
unet_train_X = np.array(unet_train_X)
unet_train_Y = np.array(unet_train_Y)
print('Training done')

### Training the model

In [ ]:
num_epoch = 60
batch_size = 30
n_points = len(unet_train_X)

history = seq_model.fit(x=unet_train_X, y=unet_train_Y, 
                validation_split=0.15,
                epochs=num_epoch,steps_per_epoch = np.ceil(n_points / batch_size), callbacks =[es,mc],  shuffle = True)

In [ ]:
seq_model.predict(unet_train_X[:2])

In [ ]:
## Gradcam

In [ ]:
!pip install timm

In [ ]:
import os
import cv2
import timm
import random
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Function
from torchvision import transforms

plt.rcParams["figure.figsize"] = (20,20)

In [ ]:
class FeatureExtractor():
    """ Class for extracting activations and
    registering gradients from targetted intermediate layers """

    def __init__(self, model, target_layers):
        self.model = model
        self.target_layers = target_layers
        self.gradients = []

    def save_gradient(self, grad):
        self.gradients.append(grad)

    def __call__(self, x):
        outputs = []
        self.gradients = []
        for name, module in self.model._modules.items():
            x = module(x)
            if name in self.target_layers:
                x.register_hook(self.save_gradient)
                outputs += [x]
        return outputs, x

class ModelOutputs():
    """ Class for making a forward pass, and getting:
    1. The network output.
    2. Activations from intermeddiate targetted layers.
    3. Gradients from intermeddiate targetted layers. """

    def __init__(self, model, feature_module, target_layers):
        self.model = model
        self.feature_module = feature_module
        self.feature_extractor = FeatureExtractor(self.feature_module, target_layers)

    def get_gradients(self):
        return self.feature_extractor.gradients

    def __call__(self, x):
        target_activations = []
        for name, module in self.model._modules.items():
            if module == self.feature_module:
                target_activations, x = self.feature_extractor(x)
            elif "avgpool" in name.lower():
                x = module(x)
                x = x.view(x.size(0),-1)
            else:
                x = module(x)

        return target_activations, x

def preprocess_image(img):
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    preprocessing = transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])
    return preprocessing(img.copy()).unsqueeze(0)

def show_cam_on_image(img, mask):
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    cam = heatmap + np.float32(img)
    cam = cam / np.max(cam)
    return np.uint8(255 * cam)

class GradCam:
    def __init__(self, model, feature_module, target_layer_names, use_cuda):
        self.model = model
        self.feature_module = feature_module
        self.model.eval()
        self.cuda = use_cuda
        if self.cuda:
            self.model = model.cuda()

        self.extractor = ModelOutputs(self.model, self.feature_module, target_layer_names)

    def forward(self, input_img):
        return self.model(input_img)

    def __call__(self, input_img, target_category=None):
        if self.cuda:
            input_img = input_img.cuda()

        features, output = self.extractor(input_img)

        if target_category == None:
            target_category = np.argmax(output.cpu().data.numpy())

        one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
        one_hot[0][target_category] = 1
        one_hot = torch.from_numpy(one_hot).requires_grad_(True)
        if self.cuda:
            one_hot = one_hot.cuda()
        
        one_hot = torch.sum(one_hot * output)

        self.feature_module.zero_grad()
        self.model.zero_grad()
        one_hot.backward(retain_graph=True)

        grads_val = self.extractor.get_gradients()[-1].cpu().data.numpy()

        target = features[-1]
        target = target.cpu().data.numpy()[0, :]

        weights = np.mean(grads_val, axis=(2, 3))[0, :]
        cam = np.zeros(target.shape[1:], dtype=np.float32)

        for i, w in enumerate(weights):
            cam += w * target[i, :, :]

        cam = np.maximum(cam, 0)
        cam = cv2.resize(cam, input_img.shape[2:])
        cam = cam - np.min(cam)
        cam = cam / np.max(cam)
        return cam


class GuidedBackpropReLU(Function):
    @staticmethod
    def forward(self, input_img):
        positive_mask = (input_img > 0).type_as(input_img)
        output = torch.addcmul(torch.zeros(input_img.size()).type_as(input_img), input_img, positive_mask)
        self.save_for_backward(input_img, output)
        return output

    @staticmethod
    def backward(self, grad_output):
        input_img, output = self.saved_tensors
        grad_input = None

        positive_mask_1 = (input_img > 0).type_as(grad_output)
        positive_mask_2 = (grad_output > 0).type_as(grad_output)
        grad_input = torch.addcmul(torch.zeros(input_img.size()).type_as(input_img),
                                   torch.addcmul(torch.zeros(input_img.size()).type_as(input_img), grad_output,
                                                 positive_mask_1), positive_mask_2)
        return grad_input


class GuidedBackpropReLUModel:
    def __init__(self, model, use_cuda):
        self.model = model
        self.model.eval()
        self.cuda = use_cuda
        if self.cuda:
            self.model = model.cuda()

        def recursive_relu_apply(module_top):
            for idx, module in module_top._modules.items():
                recursive_relu_apply(module)
                if module.__class__.__name__ == 'ReLU':
                    module_top._modules[idx] = GuidedBackpropReLU.apply

        # replace ReLU with GuidedBackpropReLU
        recursive_relu_apply(self.model)

    def forward(self, input_img):
        return self.model(input_img)

    def __call__(self, input_img, target_category=None):
        if self.cuda:
            input_img = input_img.cuda()

        input_img = input_img.requires_grad_(True)

        output = self.forward(input_img)

        if target_category == None:
            target_category = np.argmax(output.cpu().data.numpy())

        one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
        one_hot[0][target_category] = 1
        one_hot = torch.from_numpy(one_hot).requires_grad_(True)
        if self.cuda:
            one_hot = one_hot.cuda()

        one_hot = torch.sum(one_hot * output)
        one_hot.backward(retain_graph=True)

        output = input_img.grad.cpu().data.numpy()
        output = output[0, :, :, :]

        return output

def deprocess_image(img):
    """ see https://github.com/jacobgil/keras-grad-cam/blob/master/grad-cam.py#L65 """
    img = img - np.mean(img)
    img = img / (np.std(img) + 1e-5)
    img = img * 0.1
    img = img + 0.5
    img = np.clip(img, 0, 1)
    return np.uint8(img*255)


In [ ]:
""" 
1. Loads an image with opencv.
2. Preprocesses it and converts to a pytorch variable.
3. Makes a forward pass to find the category index with the highest score,
and computes intermediate activations.
Makes the visualization. 

"""
from PIL import Image
IMAGE_PATH = "../input/prostate-cancer-grade-assessment/train_images/001c62abd11fa4b57bf7a6c603a11bb9.tiff"
im = Image.open("../input/prostate-cancer-grade-assessment/train_images/001c62abd11fa4b57bf7a6c603a11bb9.tiff")



img = np.float32(img) / 255
# # Opencv loads as BGR:
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
input_img = preprocess_image(img)

grad_cam = GradCam(model=seq_model, feature_module=model.blocks,
                   target_layer_names=["conv2d_13"], use_cuda=use_cuda)

# If None, returns the map for the highest scoring category.
# Otherwise, targets the requested category.
target_category = None
grayscale_cam = grad_cam(input_img, target_category)

grayscale_cam = cv2.resize(grayscale_cam, (img.shape[1], img.shape[0]))
cam = show_cam_on_image(img, grayscale_cam)

gb_model = GuidedBackpropReLUModel(model=seq_model, use_cuda=use_cuda)
gb = gb_model(input_img, target_category=target_category)
gb = gb.transpose((1, 2, 0))

cam_mask = cv2.merge([grayscale_cam, grayscale_cam, grayscale_cam])
cam_gb = deprocess_image(cam_mask*gb)
gb = deprocess_image(gb)

cv2.imwrite("cam.jpg", cam)
cv2.imwrite('gb.jpg', gb)
cv2.imwrite('cam_gb.jpg', cam_gb);

In [ ]:
for image in base_data['image_id']:
    if os.path.exists(BASE_DIR+'train_images/'+image+'.tiff'):
        print(BASE_DIR+'train_images/'+image+'.tiff')

# Step 2: Overlay the mask on the images

In [ ]:
def overlay_mask_on_slide(image_id, alpha=0.8, max_size=(IMG_SIZE, IMG_SIZE)):
    
    slide = openslide.OpenSlide(os.path.join(BASE_DIR+"train_images", f'{image_id}.tiff'))
    mask = openslide.OpenSlide(os.path.join(BASE_DIR+"train_label_masks", f'{image_id}_mask.tiff'))
#     mask = seq_model.predict(x=[slide])

    slide_data = slide.read_region((0,0), slide.level_count - 1, slide.level_dimensions[-1])
    mask_data = mask.read_region((0,0), mask.level_count - 1, mask.level_dimensions[-1])
    mask_data = mask_data.split()[0]

    alpha_int = int(round(255*alpha))
    alpha_content = np.less(mask_data.split()[0], 2).astype('uint8') * alpha_int + (255 - alpha_int)

    alpha_content = PIL.Image.fromarray(alpha_content)
    preview_palette = np.zeros(shape=768, dtype=int)

    preview_palette[0:18] = (np.array([0, 0, 0, 0.5, 0.5, 0.5, 0, 1, 0, 1, 1, 0.7, 1, 0.5, 0, 1, 0, 0]) * 255).astype(int)
    
    mask_data.putpalette(data=preview_palette.tolist())
    mask_rgb = mask_data.convert(mode='RGB')
    overlayed_image = PIL.Image.composite(image1=slide_data, image2=mask_rgb, mask=alpha_content)
    overlayed_image.thumbnail(size=max_size, resample=0)

    overlayed_image = overlayed_image.resize(max_size)

    return overlayed_image

In [ ]:
#Creating directories for the images
overlay_train_img_folder = OVERLAY_IMG_DIR+'train/'
overlay_test_img_folder = OVERLAY_IMG_DIR+'test/'

if not os.path.exists(overlay_train_img_folder):
    os.makedirs(overlay_train_img_folder)

if not os.path.exists(overlay_test_img_folder):
    os.makedirs(overlay_test_img_folder)

#A directory for each label
labels = unet_train_labels.unique()
for lbl in labels:
    if not os.path.exists(overlay_train_img_folder+str(lbl)):
        os.makedirs(overlay_train_img_folder+str(lbl))
    
    if not os.path.exists(overlay_test_img_folder+str(lbl)):
        os.makedirs(overlay_test_img_folder+str(lbl))

In [ ]:
for i, img_id in tqdm(enumerate(unet_train_ids)):
    overlay_image = overlay_mask_on_slide(image_id=img_id)
    file_name = overlay_train_img_folder+'/'+str(unet_train_labels.iloc[i])+'/'+img_id+'.jpeg'
    overlay_image.save(file_name)

for i, img_id in tqdm(enumerate(unet_test_ids)):
    overlay_image = overlay_mask_on_slide(image_id=img_id)
    file_name = overlay_test_img_folder+'/'+str(unet_test_labels.iloc[i])+'/'+img_id+'.jpeg'
    overlay_image.save(file_name)

# Step 3: Tiling of the overlay images

In [ ]:
tiling_train_img_data = TILING_IMG_DIR+'train/'
tiling_test_img_data = TILING_IMG_DIR+'test/'

if not os.path.exists(tiling_train_img_data):
    os.makedirs(tiling_train_img_data)

if not os.path.exists(tiling_test_img_data):
    os.makedirs(tiling_test_img_data)
    
labels = unet_train_labels.unique()
for lbl in labels:
    if not os.path.exists(tiling_train_img_data+str(lbl)):
        os.makedirs(tiling_train_img_data+str(lbl))
    
    if not os.path.exists(tiling_test_img_data+str(lbl)):
        os.makedirs(tiling_test_img_data+str(lbl))

# Parameters for cropping images
cropPx= 56
cropN = 16
assert np.sqrt(cropN) == round(np.sqrt(cropN))

In [ ]:
def tile(img):
    result = []
    shape = img.shape
    pad0,pad1 = (cropPx - shape[0]%cropPx)%cropPx, (cropPx - shape[1]%cropPx)%cropPx
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                constant_values=255)
    
    img = img.reshape(img.shape[0]//cropPx,cropPx,img.shape[1]//cropPx,cropPx,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,cropPx,cropPx,3)
    
    if len(img) < cropN:
        img = np.pad(img,[[0,cropN-len(img)],[0,0],[0,0],[0,0]],constant_values=255)
    idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:cropN]
    img = img[idxs]
    for i in range(len(img)):
        result.append({'img':img[i], 'idx':i})
    return result

In [ ]:
len(os.listdir(overlay_test_img_folder+str(label)))

In [ ]:
nbCol = int(np.sqrt(cropN))

labels = os.listdir(overlay_train_img_folder)
for label in tqdm(labels):
    for name in tqdm(os.listdir(overlay_train_img_folder+str(label))):
        img = skimage.io.MultiImage(os.path.join(overlay_train_img_folder+str(label)+'/',name))[-1]
        tiles = tile(img)
        stackImg = np.vstack([np.hstack([tiles[nbCol*col + row]['img'] for row in range(nbCol)])
                   for col in range(nbCol)])
        cv2.imwrite(tiling_train_img_data+str(label)+'/'+name, stackImg)


names = os.listdir(overlay_test_img_folder)
for label in tqdm(labels):
    for name in tqdm(os.listdir(overlay_test_img_folder+str(label))):
        img = skimage.io.MultiImage(os.path.join(overlay_test_img_folder+str(label)+'/',name))[-1]
        tiles = tile(img)
        stackImg = np.vstack([np.hstack([tiles[nbCol*col + row]['img'] for row in range(nbCol)])
                   for col in range(nbCol)])
        cv2.imwrite(tiling_test_img_data+str(label)+'/'+name, stackImg)

In [ ]:
overlay_test_img_folder+'0/'+os.listdir(overlay_test_img_folder+'0')[0]

In [ ]:
os.listdir(overlay_test_img_folder)

In [ ]:
img = Image.open(overlay_test_img_folder+'0/'+os.listdir(overlay_test_img_folder+'0')[0])
img = img.resize((IMG_SIZE, IMG_SIZE))
img = np.array(img)

In [ ]:
plt.imshow(img)

# Step 4: Classifier

### Image data Generator for the model

In [ ]:
 image_gen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    rescale=1/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

In [ ]:
batch_size = 32
train_image_gen = image_gen.flow_from_directory(tiling_train_img_data,
                                                target_size=(IMG_SIZE, IMG_SIZE),
                                                batch_size=batch_size,
                                                class_mode="categorical")

In [ ]:
test_image_gen = image_gen.flow_from_directory(tiling_test_img_data,
                                                target_size=(IMG_SIZE, IMG_SIZE),
                                                batch_size=batch_size,
                                                class_mode="categorical")

### The classifier model - Xception

In [ ]:
model_builder = keras.applications.xception.Xception

model = models.Sequential()
model.add(model_builder(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3), pooling='avg'))
model.add(Dense(6, activation='softmax'))

# optimizer=keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=0.001, decay=0.0, amsgrad=True)

model.compile(
    loss="categorical_crossentropy",
    optimizer=optimizers.RMSprop(lr=3e-4),
    metrics=["acc"],
)

In [ ]:
from keras.callbacks import EarlyStopping,ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)
mc = ModelCheckpoint(SAVE_DIR+'Classifier_checkpoint.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

In [ ]:
NUMBER_OF_TRAINING_IMAGES = unet_train_ids.shape[0]
NUMBER_OF_TESTING_IMAGES = unet_test_ids.shape[0]
results = model.fit(
    train_image_gen,
    steps_per_epoch=NUMBER_OF_TRAINING_IMAGES // batch_size,
    epochs=50,
    validation_data=test_image_gen,
    validation_steps=NUMBER_OF_TESTING_IMAGES // batch_size,
    verbose=1,
    use_multiprocessing=True,
    callbacks =[es,mc],
    workers=4,
)

In [ ]:
img = Image.open(tiling_test_img_data+'2/'+os.listdir(overlay_test_img_folder+'2')[1])
img = img.resize((IMG_SIZE, IMG_SIZE))
img = np.array(img)

img = np.expand_dims(img, axis=0)
model.predict(img)

In [ ]:
img.shape

In [ ]:
model.summary()